In [ ]:
import numpy as np
import os
from simple_cysh import * # includes pandas as pd

In [ ]:
program_df = get_cysh_df('Program__c', ['Id', 'Name'], 
                         rename_id=True)
program_df.rename(columns={'Name': 'Program'},inplace=True)

# get sections, merge in program name, filter for sections of interest
section_df = get_cysh_df('Section__c', ['Id', 'School__c', 'Program__c', 'In_After_School__c'], 
                         rename_id=True, 
#                          where="Active__c = True"
                        )
section_df = section_df.merge(program_df, on='Program__c', how='left')
section_df = section_df.loc[section_df['Program'].isin([
    'Coaching: Attendance', 
    'Tutoring: Literacy', 
    'Tutoring: Math',
    'SEL Check In Check Out',
])]

stu_sec_df = get_cysh_df('Student_Section__c', ['Id', 'Section__c', 'Student__c', 'Student_Id__c', 
                                                'Student_Name__c', 'Student_Grade__c', 
                                                'Amount_of_Time__c', 'Intervention_Enrollment_Start_Date__c', 
                                                'Enrollment_End_Date__c', 'Section_Exit_Reason__c'],
                         rename_id=True, 
#                          where="Active__c = True"
                        )
stu_sec_df = stu_sec_df.loc[~stu_sec_df['Student_Name__c'].str.contains('DELETE')]
stu_lastnames = get_cysh_df('Student__c', ['Id', 'Student_Last_Name__c'], rename_id=True)
stu_sec_df = stu_sec_df.merge(stu_lastnames, how='left', on='Student__c')

# Get IA's for each student, and include Program to match with sections on Student_Program
stud_ia_df = get_cysh_df('Indicator_Area_Student__c', ['Id', 'Student__c', 'Indicator_Area__c'], 
                         rename_id=True, 
#                          where="Active__c = True"
                        )
ia_df = get_cysh_df('Indicator_Area__c', ['Id', 'Indicator_Area_Type__c'], rename_id=True)
ia_df['Program'] = ia_df['Indicator_Area_Type__c'].map({'Attendance':'Coaching: Attendance',
                                                        'Behavior':'SEL Check In Check Out',
                                                        'Math':'Tutoring: Math',
                                                        'ELA/Literacy':'Tutoring: Literacy'
                                                       })

ia_df = ia_df.merge(program_df, how='left', on='Program')
stud_ia_df = stud_ia_df.merge(ia_df, on='Indicator_Area__c')
stud_ia_df['Student_Program'] = stud_ia_df['Student__c'] + "_" + stud_ia_df['Program__c']

school_df = get_cysh_df('Account', ['Id', 'Name'])
school_df.rename(columns={'Id':'School__c', 'Name':'School'}, inplace=True)

In [ ]:
# merges
df = section_df.merge(school_df, how='left', on='School__c')
df = df.merge(stu_sec_df, on='Section__c')
df['Student_Program'] = df['Student__c'] + "_" + df['Program__c']
# filter out extended learning after ToT sum
df = df.loc[df['In_After_School__c']!='Extended Learning']
df = df.merge(stud_ia_df[['Student_Program', 'Indicator_Area__c', 'Indicator_Area_Type__c']], how='left', on='Student_Program')
df.drop_duplicates('Student_Program', inplace=True)
# filter for IAs
df = df.loc[~df['Indicator_Area__c'].isnull()]

In [ ]:

df[['Student__c', 'Indicator_Area_Type__c']].pivot_table(index='Student__c', columns='Indicator_Area_Type__c', aggfunc=len, fill_value=np.nan)

In [ ]:
miri = pd.read_csv('Z:\ChiPrivate\Chicago Reports and Evaluation\SY18\MIRI SAM Exports\SAM_EXP_CSV_20180522152820906.csv')

cols = [
    'SIS_ID',
    'SMI_TEST_DATE_1',
    'SMI_LAST_QUANTILE_DATE',
    'SRI_TEST_DATE_1',
    'SRI_LAST_LEXILE_DATE',
]

miri = miri[cols].copy()
miri['SMI_LAST_QUANTILE_DATE'] = pd.to_datetime(miri['SMI_LAST_QUANTILE_DATE'])
miri['SRI_LAST_LEXILE_DATE'] = pd.to_datetime(miri['SRI_LAST_LEXILE_DATE'])

In [ ]:
df = df.merge(miri, left_on='Student_Id__c', right_on='SIS_ID', how='left')

df.loc[(df['Program'] == 'Tutoring: Math')
       & ~df['SMI_TEST_DATE_1'].isnull()
       & (df['SMI_LAST_QUANTILE_DATE'] < '04-01-2018'), 'No_Spring_MIRI'] = True
df.loc[(df['Program'] == 'Tutoring: Literacy')
       & ~df['SRI_TEST_DATE_1'].isnull()
       & (df['SRI_LAST_LEXILE_DATE'] < '04-01-2018'), 'No_Spring_MIRI'] = True

df = df.loc[df['No_Spring_MIRI'] == True]

df = df.loc[~df['School'].str.contains('Collins')]

In [ ]:
df[['School', 'Student_Id__c', 'Program', 'SMI_TEST_DATE_1', 'SMI_LAST_QUANTILE_DATE', 'SRI_TEST_DATE_1', 'SRI_LAST_LEXILE_DATE']].to_excel('Z:\ChiPrivate\Chicago Reports and Evaluation\SY18\Students Missing Spring MIRI.xlsx', index=False)